In [2]:
import pandas as pd

In [3]:
import re

In [15]:
import numpy as np

In [37]:
timesData = pd.read_csv("dataframe/timesData.csv")
timesData.head()

,world_rank,university_name,country,teaching,international,research,citations,income,total_score,num_students,student_staff_ratio,international_students,female_male_ratio,year
0,1,Harvard University,United States of America,99.7,72.4,98.7,98.8,34.5,96.1,"20,152",8.9,25%,NaN,2011
1,2,California Institute of Technology,United States of America,97.7,54.6,98.0,99.9,83.7,96.0,"2,243",6.9,27%,33 : 67,2011
2,3,Massachusetts Institute of Technology,United States of America,97.8,82.3,91.4,99.9,87.5,95.6,"11,074",9.0,33%,37 : 63,2011
3,4,Stanford University,United States of America,98.3,29.5,98.1,99.2,64.3,94.3,"15,596",7.8,22%,42 : 58,2011
4,5,Princeton University,United States of America,90.9,70.3,95.4,99.9,-,94.2,"7,929",8.4,27%,45 : 55,2011


In [57]:
timesData.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2603 entries, 0 to 2602
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   world_rank              2603 non-null   object 
 1   university_name         2603 non-null   object 
 2   country                 2603 non-null   object 
 3   teaching                2603 non-null   float64
 4   international           2603 non-null   object 
 5   research                2603 non-null   float64
 6   citations               2603 non-null   float64
 7   income                  2385 non-null   float64
 8   total_score             2603 non-null   object 
 9   num_students            2544 non-null   object 
 10  student_staff_ratio     2544 non-null   float64
 11  international_students  2536 non-null   object 
 12  female_male_ratio       2370 non-null   object 
 13  year                    2603 non-null   int64  
dtypes: float64(5), int64(1), object(8)
memor

In [42]:
pd.read_csv?

In [56]:
timesData['income']=pd.to_numeric(timesData['income'], errors='coerce')

In [ ]:
shanghaiData = pd.read_csv("dataframe/shanghaiData.csv")
shanghaiData.head()

In [ ]:
school_and_country = pd.read_csv("dataframe/school_and_country_table.csv")
school_and_country.head()

In [ ]:
cwurData = pd.read_csv("dataframe/cwurData.csv")
cwurData.head()

In [ ]:
educational_attainment = pd.read_csv("dataframe/educational_attainment_supplementary_data.csv")
educational_attainment.head()

In [ ]:
education_expenditure = pd.read_csv("dataframe/education_expenditure_supplementary_data.csv",engine='python')
education_expenditure.head()

1. For each university, extract from the times dataset the most recent and the least recent data, obtaining two separate dataframe

In [57]:
most_recent_data=timesData.loc[(timesData.groupby('university_name')['year'].idxmax())]
most_recent_data.head()

,world_rank,university_name,country,teaching,international,research,citations,income,total_score,num_students,student_staff_ratio,international_students,female_male_ratio,year
2405,601-800,AGH University of Science and Technology,Poland,14.2,17.9,3.7,35.7,-,-,"35,569",17.0,1%,-,2016
2003,201-250,Aalborg University,Denmark,25.1,71.0,28.4,73.8,43.7,-,"17,422",15.9,15%,48 : 52,2016
2056,251-300,Aalto University,Finland,31.1,65.4,32.8,62.1,61.6,-,"16,099",24.2,17%,32 : 68,2016
1908,=106,Aarhus University,Denmark,36.9,76.8,50.7,79.8,68.3,57.7,"23,895",13.6,14%,54 : 46,2016
2105,301-350,Aberystwyth University,United Kingdom,21.6,72.2,18.9,67.2,31.3,-,"9,252",19.2,18%,48 : 52,2016


In [59]:
least_recent_data=timesData.loc[(timesData.groupby('university_name')['year'].idxmin())]
least_recent_data.head()

,world_rank,university_name,country,teaching,international,research,citations,income,total_score,num_students,student_staff_ratio,international_students,female_male_ratio,year
2405,601-800,AGH University of Science and Technology,Poland,14.2,17.9,3.7,35.7,NaN,-,"35,569",17.0,1%,-,2016
501,301-350,Aalborg University,Denmark,19.0,75.3,20.0,27.1,36.4,-,"17,422",15.9,15%,48 : 52,2012
502,301-350,Aalto University,Finland,26.2,49.0,22.2,37.5,61.9,-,"16,099",24.2,17%,32 : 68,2012
166,167,Aarhus University,Denmark,38.1,33.4,55.6,57.3,61.5,49.9,"23,895",13.6,14%,54 : 46,2011
476,276-300,Aberystwyth University,United Kingdom,19.8,63.8,15.5,56.6,35.5,-,"9,252",19.2,18%,48 : 52,2012


In [28]:
##timesData[timesData['year']>=timesData['year'].max()]

In [ ]:
##timesData[timesData['university_name']=='Aalto University']

In [ ]:
#timesData.groupby('university_name').min()['year']


In [ ]:
#timesData.groupby('university_name').max()['year']

2. For each university, compute the improvement in "income" between the least recent and the most recent data points

In [60]:
punto2=pd.merge(most_recent_data, least_recent_data, on='university_name')
punto2.head()

,world_rank_x,university_name,country_x,teaching_x,international_x,research_x,citations_x,income_x,total_score_x,num_students_x,...,international_y,research_y,citations_y,income_y,total_score_y,num_students_y,student_staff_ratio_y,international_students_y,female_male_ratio_y,year_y
0,601-800,AGH University of Science and Technology,Poland,14.2,17.9,3.7,35.7,NaN,-,"35,569",...,17.9,3.7,35.7,NaN,-,"35,569",17.0,1%,-,2016
1,201-250,Aalborg University,Denmark,25.1,71.0,28.4,73.8,43.7,-,"17,422",...,75.3,20.0,27.1,36.4,-,"17,422",15.9,15%,48 : 52,2012
2,251-300,Aalto University,Finland,31.1,65.4,32.8,62.1,61.6,-,"16,099",...,49.0,22.2,37.5,61.9,-,"16,099",24.2,17%,32 : 68,2012
3,=106,Aarhus University,Denmark,36.9,76.8,50.7,79.8,68.3,57.7,"23,895",...,33.4,55.6,57.3,61.5,49.9,"23,895",13.6,14%,54 : 46,2011
4,301-350,Aberystwyth University,United Kingdom,21.6,72.2,18.9,67.2,31.3,-,"9,252",...,63.8,15.5,56.6,35.5,-,"9,252",19.2,18%,48 : 52,2012


In [ ]:
punto2.columns

In [32]:
##nuovo_punto2=punto2[punto2[['income_x','income_y']].notnull()]
##nuovo_punto2.head()

In [ ]:
##def calcola_perc(riga):
    ##return riga['income_x'] - riga['income_y']

In [ ]:
##nuovo_punto2['income_improvement']=nuovo_punto2.apply(calcola_perc, axis=1)
##nuovo_punto2.head()

In [ ]:
##nuovo_punto2['differenza']=nuovo_punto2['income_x']-nuovo_punto2['income_y']
##nuovo_punto2

In [ ]:
#nuovo_punto2[nuovo_punto2['income_y']-nuovo_punto2['income_x']]

In [35]:
##lista=nuovo_punto2['income_x']
##lista


In [36]:
##nuovo=lista.astype(float)
##nuovo

In [ ]:
def calcola_perc(riga):
    return riga['votes_first_vote']/ risultato[riga['state']] *100

In [ ]:
##nuovo_punto2['income_y']=nuovo_punto2['income_y'].to_numeric()

In [ ]:
##nuovo_punto2=np.array(list, dtype=float)

In [65]:
punto2['income_improvement']=punto2['income_y']-punto2['income_x']
punto2.head()

,world_rank_x,university_name,country_x,teaching_x,international_x,research_x,citations_x,income_x,total_score_x,num_students_x,...,research_y,citations_y,income_y,total_score_y,num_students_y,student_staff_ratio_y,international_students_y,female_male_ratio_y,year_y,income_improvement
0,601-800,AGH University of Science and Technology,Poland,14.2,17.9,3.7,35.7,NaN,-,"35,569",...,3.7,35.7,NaN,-,"35,569",17.0,1%,-,2016,NaN
1,201-250,Aalborg University,Denmark,25.1,71.0,28.4,73.8,43.7,-,"17,422",...,20.0,27.1,36.4,-,"17,422",15.9,15%,48 : 52,2012,-7.3
2,251-300,Aalto University,Finland,31.1,65.4,32.8,62.1,61.6,-,"16,099",...,22.2,37.5,61.9,-,"16,099",24.2,17%,32 : 68,2012,0.3
3,=106,Aarhus University,Denmark,36.9,76.8,50.7,79.8,68.3,57.7,"23,895",...,55.6,57.3,61.5,49.9,"23,895",13.6,14%,54 : 46,2011,-6.8
4,301-350,Aberystwyth University,United Kingdom,21.6,72.2,18.9,67.2,31.3,-,"9,252",...,15.5,56.6,35.5,-,"9,252",19.2,18%,48 : 52,2012,4.2


3. Find the university with the largest increase computed in the previous point

In [68]:
punto2.loc[punto2['income_improvement']>= punto2['income_improvement'].max()]['university_name'] #possiamo anche togliere l'ultima quadra


414    State University of New York Albany
Name: university_name, dtype: object

4. For each ranking, consider only the most recent data point. For each university, compute the maximum difference between the rankings (e.g. for Aarhus University the value is 122-73=49). Notice that some rankings are expressed as a range

In [63]:
cwurData[cwurData['institution']=='Aarhus University']

,world_rank,institution,country,national_rank,quality_of_education,alumni_employment,quality_of_faculty,publications,influence,citations,broad_impact,patents,score,year
318,119,Aarhus University,Denmark,2,292,188,107,90,78,81,90.0,194,50.17,2014
1321,122,Aarhus University,Denmark,2,310,165,106,91,81,101,95.0,154,49.65,2015


In [ ]:
shanghaiData[shanghaiData['university_name']=='Aarhus University']

5. Consider only the most recent data point of the times dataset. Compute the number of male and female students for each country.

In [39]:
most_recent_data.head()

,world_rank,university_name,country,teaching,international,research,citations,income,total_score,num_students,student_staff_ratio,international_students,female_male_ratio,year
2405,601-800,AGH University of Science and Technology,Poland,14.2,17.9,3.7,35.7,-,-,"35,569",17.0,1%,-,2016
2003,201-250,Aalborg University,Denmark,25.1,71.0,28.4,73.8,43.7,-,"17,422",15.9,15%,48 : 52,2016
2056,251-300,Aalto University,Finland,31.1,65.4,32.8,62.1,61.6,-,"16,099",24.2,17%,32 : 68,2016
1908,=106,Aarhus University,Denmark,36.9,76.8,50.7,79.8,68.3,57.7,"23,895",13.6,14%,54 : 46,2016
2105,301-350,Aberystwyth University,United Kingdom,21.6,72.2,18.9,67.2,31.3,-,"9,252",19.2,18%,48 : 52,2016


In [ ]:
most_recent_data['year']

In [ ]:
most_recent_datab

In [58]:
rapporto=most_recent_data['female_male_ratio']
rapporto.head()

2405          -
2003    48 : 52
2056    32 : 68
1908    54 : 46
2105    48 : 52
Name: female_male_ratio, dtype: object

In [59]:
dataframe=pd.DataFrame(rapporto)
dataframe.head()

,female_male_ratio
2405,-
2003,48 : 52
2056,32 : 68
1908,54 : 46
2105,48 : 52


In [46]:
most_recent_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 818 entries, 2405 to 2134
Data columns (total 16 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   world_rank              818 non-null    object 
 1   university_name         818 non-null    object 
 2   country                 818 non-null    object 
 3   teaching                818 non-null    float64
 4   international           818 non-null    object 
 5   research                818 non-null    float64
 6   citations               818 non-null    float64
 7   income                  818 non-null    object 
 8   total_score             818 non-null    object 
 9   num_students            2 non-null      float64
 10  student_staff_ratio     796 non-null    float64
 11  international_students  793 non-null    object 
 12  female_male_ratio       742 non-null    object 
 13  year                    818 non-null    int64  
 14  female                  739 non-null  

In [61]:
most_recent_data['female'] = most_recent_data['female_male_ratio'].str.extract(r'(^\d+)')
most_recent_data['male'] = most_recent_data['female_male_ratio'].str.extract(r'(\d+$)')
most_recent_data.head()

,world_rank,university_name,country,teaching,international,research,citations,income,total_score,num_students,student_staff_ratio,international_students,female_male_ratio,year,female,male
2405,601-800,AGH University of Science and Technology,Poland,14.2,17.9,3.7,35.7,-,-,"35,569",17.0,1%,-,2016,NaN,NaN
2003,201-250,Aalborg University,Denmark,25.1,71.0,28.4,73.8,43.7,-,"17,422",15.9,15%,48 : 52,2016,48,52
2056,251-300,Aalto University,Finland,31.1,65.4,32.8,62.1,61.6,-,"16,099",24.2,17%,32 : 68,2016,32,68
1908,=106,Aarhus University,Denmark,36.9,76.8,50.7,79.8,68.3,57.7,"23,895",13.6,14%,54 : 46,2016,54,46
2105,301-350,Aberystwyth University,United Kingdom,21.6,72.2,18.9,67.2,31.3,-,"9,252",19.2,18%,48 : 52,2016,48,52


In [63]:
#most_recent_data['num_students']=pd.to_numeric(most_recent_data['num_students'], errors='coerce')
most_recent_data['male']=pd.to_numeric(most_recent_data['male'], errors='coerce')
most_recent_data['female']=pd.to_numeric(most_recent_data['female'], errors='coerce')

In [89]:
most_recent_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 818 entries, 2405 to 2134
Data columns (total 16 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   world_rank              818 non-null    object 
 1   university_name         818 non-null    object 
 2   country                 818 non-null    object 
 3   teaching                818 non-null    float64
 4   international           818 non-null    object 
 5   research                818 non-null    float64
 6   citations               818 non-null    float64
 7   income                  818 non-null    object 
 8   total_score             818 non-null    object 
 9   num_students            796 non-null    float64
 10  student_staff_ratio     796 non-null    float64
 11  international_students  793 non-null    object 
 12  female_male_ratio       742 non-null    object 
 13  year                    818 non-null    int64  
 14  female                  739 non-null  

In [64]:
most_recent_data.head()

,world_rank,university_name,country,teaching,international,research,citations,income,total_score,num_students,student_staff_ratio,international_students,female_male_ratio,year,female,male
2405,601-800,AGH University of Science and Technology,Poland,14.2,17.9,3.7,35.7,-,-,"35,569",17.0,1%,-,2016,NaN,NaN
2003,201-250,Aalborg University,Denmark,25.1,71.0,28.4,73.8,43.7,-,"17,422",15.9,15%,48 : 52,2016,48.0,52.0
2056,251-300,Aalto University,Finland,31.1,65.4,32.8,62.1,61.6,-,"16,099",24.2,17%,32 : 68,2016,32.0,68.0
1908,=106,Aarhus University,Denmark,36.9,76.8,50.7,79.8,68.3,57.7,"23,895",13.6,14%,54 : 46,2016,54.0,46.0
2105,301-350,Aberystwyth University,United Kingdom,21.6,72.2,18.9,67.2,31.3,-,"9,252",19.2,18%,48 : 52,2016,48.0,52.0


In [81]:
most_recent_data['num_students'] = most_recent_data['num_students'].str.replace(',','')
most_recent_data.head()

,world_rank,university_name,country,teaching,international,research,citations,income,total_score,num_students,student_staff_ratio,international_students,female_male_ratio,year,female,male
2405,601-800,AGH University of Science and Technology,Poland,14.2,17.9,3.7,35.7,-,-,35569,17.0,1%,-,2016,NaN,NaN
2003,201-250,Aalborg University,Denmark,25.1,71.0,28.4,73.8,43.7,-,17422,15.9,15%,48 : 52,2016,48.0,52.0
2056,251-300,Aalto University,Finland,31.1,65.4,32.8,62.1,61.6,-,16099,24.2,17%,32 : 68,2016,32.0,68.0
1908,=106,Aarhus University,Denmark,36.9,76.8,50.7,79.8,68.3,57.7,23895,13.6,14%,54 : 46,2016,54.0,46.0
2105,301-350,Aberystwyth University,United Kingdom,21.6,72.2,18.9,67.2,31.3,-,9252,19.2,18%,48 : 52,2016,48.0,52.0


In [84]:
most_recent_data['num_students']=pd.to_numeric(most_recent_data['num_students'], errors='coerce')
most_recent_data.head()

,world_rank,university_name,country,teaching,international,research,citations,income,total_score,num_students,student_staff_ratio,international_students,female_male_ratio,year,female,male
2405,601-800,AGH University of Science and Technology,Poland,14.2,17.9,3.7,35.7,-,-,35569.0,17.0,1%,-,2016,NaN,NaN
2003,201-250,Aalborg University,Denmark,25.1,71.0,28.4,73.8,43.7,-,17422.0,15.9,15%,48 : 52,2016,48.0,52.0
2056,251-300,Aalto University,Finland,31.1,65.4,32.8,62.1,61.6,-,16099.0,24.2,17%,32 : 68,2016,32.0,68.0
1908,=106,Aarhus University,Denmark,36.9,76.8,50.7,79.8,68.3,57.7,23895.0,13.6,14%,54 : 46,2016,54.0,46.0
2105,301-350,Aberystwyth University,United Kingdom,21.6,72.2,18.9,67.2,31.3,-,9252.0,19.2,18%,48 : 52,2016,48.0,52.0


In [88]:
val_nonnulli=most_recent_data[(most_recent_data['male'].notnull()) | (most_recent_data['female'].notnull()) | (most_recent_data['num_students'].notnull())]
val_nonnulli.head()

,world_rank,university_name,country,teaching,international,research,citations,income,total_score,num_students,student_staff_ratio,international_students,female_male_ratio,year,female,male
2405,601-800,AGH University of Science and Technology,Poland,14.2,17.9,3.7,35.7,-,-,35569.0,17.0,1%,-,2016,NaN,NaN
2003,201-250,Aalborg University,Denmark,25.1,71.0,28.4,73.8,43.7,-,17422.0,15.9,15%,48 : 52,2016,48.0,52.0
2056,251-300,Aalto University,Finland,31.1,65.4,32.8,62.1,61.6,-,16099.0,24.2,17%,32 : 68,2016,32.0,68.0
1908,=106,Aarhus University,Denmark,36.9,76.8,50.7,79.8,68.3,57.7,23895.0,13.6,14%,54 : 46,2016,54.0,46.0
2105,301-350,Aberystwyth University,United Kingdom,21.6,72.2,18.9,67.2,31.3,-,9252.0,19.2,18%,48 : 52,2016,48.0,52.0


In [90]:
def calc_numero(row):
    
    return most_recent_data['num_students']* most_recent_data[row]//100



In [91]:
most_recent_data['num_male']=most_recent_data['male'].apply(calc_numero,axis=1)

TypeError: calc_numero() got an unexpected keyword argument 'axis'

6. Find the universities where the ratio between female and male is below the average ratio (computed over all universities)

7. For each country, compute the fraction of the students in the country that are in one of the universities computed in the previous point (that is, the denominator of the ratio is the total number of students over all universities in the country).

8. Read the file educational_attainment_supplementary_data.csv, discarding any row with missing country_name or series_name

In [26]:
educational_attainment = pd.read_csv("dataframe/educational_attainment_supplementary_data.csv")
educational_attainment.head()

,country_name,series_name,1985,1986,1987,1990,1991,1992,1993,1995,...,2005,2006,2007,2008,2009,2010,2011,2012,2013,2015
0,Afghanistan,"Barro-Lee: Average years of primary schooling,...",0.33,NaN,NaN,0.44,NaN,NaN,NaN,0.57,...,0.86,NaN,NaN,NaN,NaN,1.27,NaN,NaN,NaN,NaN
1,Afghanistan,"Barro-Lee: Average years of primary schooling,...",1.03,NaN,NaN,1.26,NaN,NaN,NaN,1.54,...,2.18,NaN,NaN,NaN,NaN,2.64,NaN,NaN,NaN,NaN
2,Afghanistan,"Barro-Lee: Average years of primary schooling,...",0.83,NaN,NaN,0.95,NaN,NaN,NaN,1.26,...,1.01,NaN,NaN,NaN,NaN,2.45,NaN,NaN,NaN,NaN
3,Afghanistan,"Barro-Lee: Average years of primary schooling,...",2.34,NaN,NaN,2.22,NaN,NaN,NaN,2.37,...,2.26,NaN,NaN,NaN,NaN,3.55,NaN,NaN,NaN,NaN
4,Afghanistan,"Barro-Lee: Average years of primary schooling,...",0.54,NaN,NaN,0.92,NaN,NaN,NaN,0.94,...,2.00,NaN,NaN,NaN,NaN,1.29,NaN,NaN,NaN,NaN


In [27]:
#educational_attainment.dropna(educational_attainment,how="any", thresh=None, subset=(['country_name','series_name']), inplace=True)
#educational_attainment.head() 

In [28]:
educ = educational_attainment[(educational_attainment['country_name'].notnull()) | (educational_attainment['series_name'].notnull())]
educ.head()

,country_name,series_name,1985,1986,1987,1990,1991,1992,1993,1995,...,2005,2006,2007,2008,2009,2010,2011,2012,2013,2015
0,Afghanistan,"Barro-Lee: Average years of primary schooling,...",0.33,NaN,NaN,0.44,NaN,NaN,NaN,0.57,...,0.86,NaN,NaN,NaN,NaN,1.27,NaN,NaN,NaN,NaN
1,Afghanistan,"Barro-Lee: Average years of primary schooling,...",1.03,NaN,NaN,1.26,NaN,NaN,NaN,1.54,...,2.18,NaN,NaN,NaN,NaN,2.64,NaN,NaN,NaN,NaN
2,Afghanistan,"Barro-Lee: Average years of primary schooling,...",0.83,NaN,NaN,0.95,NaN,NaN,NaN,1.26,...,1.01,NaN,NaN,NaN,NaN,2.45,NaN,NaN,NaN,NaN
3,Afghanistan,"Barro-Lee: Average years of primary schooling,...",2.34,NaN,NaN,2.22,NaN,NaN,NaN,2.37,...,2.26,NaN,NaN,NaN,NaN,3.55,NaN,NaN,NaN,NaN
4,Afghanistan,"Barro-Lee: Average years of primary schooling,...",0.54,NaN,NaN,0.92,NaN,NaN,NaN,0.94,...,2.00,NaN,NaN,NaN,NaN,1.29,NaN,NaN,NaN,NaN


In [24]:
educational_attainment.dropna?


9. From attainment build a dataframe with the same data, but with 4 columns: country_name, series_name, year, value

In [30]:
ed_pulito= educ.dropna(axis='columns', how='all')
ed_pulito.head()

,country_name,series_name,1985,1986,1987,1990,1991,1992,1993,1995,...,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013
0,Afghanistan,"Barro-Lee: Average years of primary schooling,...",0.33,NaN,NaN,0.44,NaN,NaN,NaN,0.57,...,NaN,0.86,NaN,NaN,NaN,NaN,1.27,NaN,NaN,NaN
1,Afghanistan,"Barro-Lee: Average years of primary schooling,...",1.03,NaN,NaN,1.26,NaN,NaN,NaN,1.54,...,NaN,2.18,NaN,NaN,NaN,NaN,2.64,NaN,NaN,NaN
2,Afghanistan,"Barro-Lee: Average years of primary schooling,...",0.83,NaN,NaN,0.95,NaN,NaN,NaN,1.26,...,NaN,1.01,NaN,NaN,NaN,NaN,2.45,NaN,NaN,NaN
3,Afghanistan,"Barro-Lee: Average years of primary schooling,...",2.34,NaN,NaN,2.22,NaN,NaN,NaN,2.37,...,NaN,2.26,NaN,NaN,NaN,NaN,3.55,NaN,NaN,NaN
4,Afghanistan,"Barro-Lee: Average years of primary schooling,...",0.54,NaN,NaN,0.92,NaN,NaN,NaN,0.94,...,NaN,2.00,NaN,NaN,NaN,NaN,1.29,NaN,NaN,NaN


In [33]:
#ed_pulito[ed_pulito['1986'].notnull()]

In [34]:
ed_pulito.transpose?

10. For each university, find the number of rankings in which they appear (it suffices to appear in one year for each ranking).

11. In the times ranking, compute the number of times each university appears

12. Find the universities that appear at most twice in the times ranking.

13. The universities that, in any year, have the same position in all three rankings (they must have the same position in a year).